# Propose BERT siamese

In [1]:
# import tensorflow as tf
import keras
# from tensorflow.python import keras
import os

Using TensorFlow backend.


In [2]:
# import tensorflow as tf
# import tensorflow.keras.backend as K_tf

# sess = K_tf.get_session()
# uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
# init_op = tf.variables_initializer(
#     [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
# )
# sess.run(init_op)

In [3]:
from __future__ import print_function, division

In [4]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [5]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras import optimizers

In [6]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [7]:
MAX_SEQUENCE_LENGTH_T = 20 # 40
MAX_SEQUENCE_LENGTH_D = 100 # 200
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

### Parse bugs preproprecessed

In [8]:
# Domain to use
DOMAIN = 'eclipse'
METHOD = 'propose_bert'
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = '{}_feature@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_feature_@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [9]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [10]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [11]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


361006

#### Dicionário de títulos e descrições

In [12]:
# !wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
# !unzip -o uncased_L-12_H-768_A-12.zip

In [13]:
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
model_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [14]:
import codecs

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

In [15]:
# import _pickle as pickle
# from collections import defaultdict
# from keras_bert import get_base_dict

# def build_vocab(baseline):
#     print('building frequency dictionaries')
#     word_freq = get_base_dict()  # A dict that contains some special tokens
#     for bug_id in tqdm(baseline.bug_ids):
#         bug = pickle.load(open(os.path.join(baseline.DIR, 'bugs', '{}.pkl'.format(bug_id)), 'rb'))
#         title_txt = "[CLS] " + bug['title'] + " [SEP]"
#         desc_txt = "[CLS] " + bug['description'] + " [SEP]"
#         for word in title_txt.split():
#             if word not in word_freq:
#                 word_freq[word] = 0
#             word_freq[word] += 1
#         for word in desc_txt.split():
#             if word not in word_freq:
#                 word_freq[word] = 0
#             word_freq[word] += 1
#     return word_freq
# token_dict = build_vocab(baseline)

In [16]:
# words = np.random.choice(range(len(token_dict.keys())), 10)
# vocab_words = list(token_dict.keys())
# for token in words:
#     print(vocab_words[token], token_dict[vocab_words[token]])

In [17]:
"Total vocabulary: {}".format(len(token_dict))

'Total vocabulary: 30522'

In [18]:
import _pickle as pickle
from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict)

def load_bugs(baseline):
    global tokenizer
    removed = []
    baseline.corpus = []
    baseline.sentence_dict = {}
    baseline.bug_set = {}
    title_padding, desc_padding = [], []
    for bug_id in tqdm(baseline.bug_ids):
        #try:
            bug = pickle.load(open(os.path.join(baseline.DIR, 'bugs', '{}.pkl'.format(bug_id)), 'rb'))
            title_txt = "[CLS] " + bug['title'] + " [SEP]"
            desc_txt = "[CLS] " + bug['description'] + " [SEP]"
            #print(title_txt, desc_txt)
            ids, segments = tokenizer.encode(title_txt, max_len=MAX_SEQUENCE_LENGTH_T)
            title_padding.append(ids)
            ids, segments = tokenizer.encode(desc_txt, max_len=MAX_SEQUENCE_LENGTH_D)
            desc_padding.append(ids)
            baseline.bug_set[bug_id] = bug
            #break
        #except:
            removed.append(bug_id)    
    # Padding
    title_padding = baseline.data_padding(title_padding, baseline.MAX_SEQUENCE_LENGTH_T)
    desc_padding = baseline.data_padding(desc_padding, baseline.MAX_SEQUENCE_LENGTH_D)
    
    for bug_id, bug_title, bug_desc in tqdm(zip(baseline.bug_ids, title_padding, desc_padding)):
            baseline.bug_set[bug_id]['title_word'] = bug_title
            baseline.bug_set[bug_id]['description_word'] = bug_desc
            bug = baseline.bug_set[bug_id]
            baseline.sentence_dict[",".join(bug_title.astype(str))] = bug['title']
            baseline.sentence_dict[",".join(bug_desc.astype(str))] = bug['description']

In [ ]:
%%time

load_bugs(baseline)
len(baseline.sentence_dict)

#### Hashing bugs by buckets

In [ ]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [21]:
%%time

experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

Reading bug ids
CPU times: user 2min 36s, sys: 4.97 ms, total: 2min 36s
Wall time: 2min 36s


#### Recovery bug ids from train

In [22]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

#### Display a random bug

In [23]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '2\n',
 'bug_status': '2\n',
 'component': '655\n',
 'creation_ts': '2011-04-18 14:16:00 -0400',
 'delta_ts': '2011-05-02 12:44:45 -0400',
 'description': 'build identifier mnemonic char is not underscore in preview mode and design view see attachment for details reproducible always steps to reproduce see attachment',
 'description_word': array([  101,  1031, 18856,  2015,  1033,  3857,  8909,  4765, 18095,
        24098, 26941,  2594, 25869,  2003,  2025,  2104,  9363,  2890,
         1999, 19236,  5549,  1998,  2640,  3193,  2156, 14449,  2005,
         4751, 16360, 14127, 21104,  2467,  4084,  2000, 21376,  2156,
        14449,  1031, 19802,  1033,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
    

### Generating the batch test

In [24]:
"Train ", len(baseline.dup_sets_train)

('Train ', 34882)

In [25]:
 # data - path
# batch_size - 128
# n_neg - 1

import random

def read_batch_bugs(self, batch, bug):
        info = np.concatenate((
            self.to_one_hot(bug['bug_severity'], self.info_dict['bug_severity']),
            self.to_one_hot(bug['bug_status'], self.info_dict['bug_status']),
            self.to_one_hot(bug['component'], self.info_dict['component']),
            self.to_one_hot(bug['priority'], self.info_dict['priority']),
            self.to_one_hot(bug['product'], self.info_dict['product']),
            self.to_one_hot(bug['version'], self.info_dict['version']))
        )
        #info.append(info_)
        batch['info'].append(info)
        batch['title'].append(bug['title_word'])
        batch['desc'].append(bug['description_word'])

def get_neg_bug(invalid_bugs, bug_ids, issues_by_buckets):
    neg_bug = random.choice(list(issues_by_buckets.keys()))
    try:
        while neg_bug in invalid_bugs or neg_bug not in issues_by_buckets:
            neg_bug = random.choice(bug_ids)
    except:
        invalid_bugs = [invalid_bugs]
        while neg_bug in invalid_bugs or neg_bug not in issues_by_buckets:
            neg_bug = random.choice(bug_ids)
    return neg_bug

def batch_iterator(baseline, data, dup_sets, bug_train_ids, batch_size, n_neg, issues_by_buckets):
    # global train_data
    # global self.dup_sets
    # global self.bug_ids
    # global self.bug_set

    random.shuffle(data)

    batch_input, batch_pos, batch_neg, master_batch_input, master_batch_neg = {'title' : [], 'desc' : [], 'info' : []}, \
                                            {'title' : [], 'desc' : [], 'info' : []}, \
                                                {'title' : [], 'desc' : [], 'info' : []},\
                                                    {'title' : [], 'desc' : [], 'info' : []}, \
                                                        {'title' : [], 'desc' : [], 'info' : []}

    n_train = len(data)

    batch_triplets = []

    for offset in range(batch_size):
        neg_bug = baseline.get_neg_bug(dup_sets[data[offset][0]], bug_train_ids)
        anchor, pos, neg = data[offset][0], data[offset][1], neg_bug
        bug_anchor = baseline.bug_set[anchor]
        bug_pos = baseline.bug_set[pos]
        bug_neg = baseline.bug_set[neg]
        # master anchor and neg
        master_anchor = baseline.bug_set[issues_by_buckets[anchor]]
        master_neg = baseline.bug_set[issues_by_buckets[neg]]
        
        baseline.read_batch_bugs(batch_input, bug_anchor)
        baseline.read_batch_bugs(batch_pos, bug_pos)
        baseline.read_batch_bugs(batch_neg, bug_neg)
        # master anchor and neg
        baseline.read_batch_bugs(master_batch_input, master_anchor)
        baseline.read_batch_bugs(master_batch_neg, master_neg)
        # triplet bug and master
        batch_triplets.append([data[offset][0], data[offset][1], neg_bug, master_anchor, master_neg])
        
        
    ids = np.full((len(batch_triplets), MAX_SEQUENCE_LENGTH_D), 0)

    batch_input['title'] = np.array(batch_input['title'])
    batch_input['desc'] = { 'token' : np.array(batch_input['desc']), 'segment' : ids }
    batch_input['info'] = np.array(batch_input['info'])
    batch_pos['title'] = np.array(batch_pos['title'])
    batch_pos['desc'] = { 'token' : np.array(batch_pos['desc']), 'segment' : ids }
    batch_pos['info'] = np.array(batch_pos['info'])
    batch_neg['title'] = np.array(batch_neg['title'])
    batch_neg['desc'] = { 'token' : np.array(batch_neg['desc']), 'segment' : ids }
    batch_neg['info'] = np.array(batch_neg['info'])
    
    # master
    master_batch_input['title'] = np.array(master_batch_input['title'])
    master_batch_input['desc'] ={ 'token' : np.array(master_batch_input['desc']), 'segment' : ids }
    master_batch_input['info'] = np.array(master_batch_input['info'])
    
    master_batch_neg['title'] = np.array(master_batch_neg['title'])
    master_batch_neg['desc'] = { 'token' : np.array(master_batch_neg['desc']), 'segment' : ids }
    master_batch_neg['info'] = np.array(master_batch_neg['info'])

    n_half = len(batch_triplets) // 2
    if n_half > 0:
        pos = np.full((1, n_half), 1)
        neg = np.full((1, n_half), 0)
        sim = np.concatenate([pos, neg], -1)[0]
    else:
        sim = np.array([np.random.choice([1, 0])])

    input_sample, input_pos, input_neg, master_input_sample, master_neg = {}, {}, {}, {}, {}

    input_sample = { 'title' : batch_input['title'], 'description' : batch_input['desc'], 'info' : batch_input['info'] }
    input_pos = { 'title' : batch_pos['title'], 'description' : batch_pos['desc'], 'info': batch_pos['info'] }
    input_neg = { 'title' : batch_neg['title'], 'description' : batch_neg['desc'], 'info': batch_neg['info'] }
    # master 
    master_input_sample = { 'title' : master_batch_input['title'], 'description' : master_batch_input['desc'], 
                           'info' : master_batch_input['info'] }
    master_neg = { 'title' : master_batch_neg['title'], 'description' : master_batch_neg['desc'], 
                           'info' : master_batch_neg['info'] }
    return batch_triplets, input_sample, input_pos, input_neg, master_input_sample, master_neg, sim #sim

### Train ids

In [26]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

In [27]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, \
                            valid_master_sample, valid_master_neg, valid_sim = batch_iterator(baseline, baseline.train_data, 
                                                                                          baseline.dup_sets_train,
                                                                                          bug_train_ids,
                                                                                          batch_size_test, 1, issues_by_buckets)

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description']['token'].shape[1]

CPU times: user 49.2 ms, sys: 0 ns, total: 49.2 ms
Wall time: 49.9 ms


In [28]:
valid_input_sample['title'].shape, valid_input_sample['description']['token'].shape, \
    valid_input_sample['info'].shape, valid_sim.shape

((128, 20), (128, 100), (128, 1682), (128,))

### Validar entrada

In [29]:
%%time 

baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5)

***Title***: jface pde color manager leaks colors
***Title***: jface jface colors should reuse colors
***Description***: every time you open and close the pde plugin manifest editor you get number new colors allocated from organization manager they are always the same number number number number number number number number number number number number number number number heres the stack trace java lang error at organization at org eclipse swt graphics device new object unknown source at org eclipse swt graphics color init unknown source at org eclipse jface resource jface colors get color setting unknown source at org eclipse jface resource jface colors get hyperlink text unknown source at org eclipse update ui forms internal form widget factory update hyperlink colors unknown source at org eclipse update ui forms internal form widget factory initialize unknown source at org eclipse update ui forms internal form widget factory init unknown source at org eclipse update ui forms internal

## Pre-trained embeddings

Loading pretrained word vectors

### Fasttext

In [30]:
vocab = baseline.load_vocabulary(os.path.join(DIR, 'vocab_embed_fasttext.pkl'))

vocabulary loaded


In [31]:
# import io

# def generating_embed(baseline, EMBED_DIR, EMBEDDING_DIM):
#     embeddings_index = {}
#     embed_path = os.path.join(EMBED_DIR, 'crawl-300d-2M.vec')
#     f = open(embed_path, 'rb')
#     f = io.open(embed_path, 'r', encoding='utf-8', newline='\n', errors='ignore')
#     n, d = map(int, f.readline().split())

#     vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed_fasttext.pkl'))
#     vocab_size = len(vocab) 

#     # Initialize uniform the vector considering the Tanh activation
#     embedding_matrix = np.random.uniform(-1.0, 1.0, (vocab_size, EMBEDDING_DIM))
#     embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

#     loop = tqdm(f)
#     loop.set_description("Loading FastText")
#     for line in loop:
#         tokens = line.rstrip().split(' ')
#         embed = list(map(float, tokens[1:]))
#         word = tokens[0]
#         embeddings_index[word] = np.asarray(embed, dtype='float32')
#         loop.update(1)
#     f.close()
#     loop.close()

#     print('Total %s word vectors in FastText 42B 300d.' % len(embeddings_index))

#     loop = tqdm(total=vocab_size)
#     loop.set_description('Loading embedding from dataset pretrained')
#     i = 0
#     for word, embed in vocab.items():
#         if word in embeddings_index:
#             embedding_matrix[i] = embeddings_index[word]
#         else:
#             embedding_matrix[i] = np.asarray(embed, dtype='float32')
#         loop.update(1)
#         i+=1
#     loop.close()
#     baseline.embedding_matrix = embedding_matrix

In [32]:
def generating_embed(baseline, EMBED_DIR, EMBEDDING_DIM):
    embeddings_index = {}
    embed_path = os.path.join(EMBED_DIR, 'glove.42B.300d.txt')
    f = open(embed_path, 'rb')
    #num_lines = sum(1 for line in open(embed_path, 'rb'))

    vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed.pkl'))
    vocab_size = len(vocab) 

    # Initialize uniform the vector considering the Tanh activation
    embedding_matrix = np.random.uniform(-1.0, 1.0, (vocab_size, EMBEDDING_DIM))
    embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

    loop = tqdm(f)
    loop.set_description("Loading Glove")
    for line in loop:
        tokens = line.split()
        word = tokens[0]
        embeddings_index[word] = np.asarray(tokens[1:], dtype='float32')
        loop.update(1)
    f.close()
    loop.close()

    print('Total %s word vectors in Glove 42B 300d.' % len(embeddings_index))

    loop = tqdm(total=vocab_size)
    loop.set_description('Loading embedding from dataset pretrained')
    i = 0
    for word, embed in vocab.items():
        if word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
        else:
            embedding_matrix[i] = np.asarray(embed, dtype='float32')
        loop.update(1)
        i+=1
    loop.close()
    baseline.embedding_matrix = embedding_matrix

In [33]:
%%time

generating_embed(baseline, EMBED_DIR=EMBED_DIR, EMBEDDING_DIM=EMBEDDING_DIM) # MAX_NB_WORDS=MAX_NB_WORDS

vocabulary loaded



Total 1917494 word vectors in Glove 42B 300d.



CPU times: user 1min 26s, sys: 3.66 s, total: 1min 30s
Wall time: 1min 28s


## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [34]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### Embedding layer

In [35]:
from keras.constraints import MaxNorm
from keras.initializers import TruncatedNormal, RandomUniform

# Is missing the padding_idx used in pytorch
# https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html
# https://stackoverflow.com/questions/54824768/rnn-model-gru-of-word2vec-to-regression-not-learning
def embedding_layer(embeddings, num_words, embedding_dim, max_sequence_length, trainable, name):
    embedding_layer = Embedding(num_words,
                                  embedding_dim,
                                  name='embedding_layer_{}'.format(name),
                                  weights=[embeddings],
                                  embeddings_constraint=MaxNorm(max_value=1, axis=0),
                                  #input_length=max_sequence_length,
                                  input_length=None,
                                  trainable=trainable)
    return embedding_layer

### BERT

https://github.com/CyberZHG/keras-bert

In [36]:
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import compile_model, get_model
from keras.layers import GlobalAveragePooling1D

def bert_model(MAX_SEQUENCE_LENGTH_D):
#     model = load_trained_model_from_checkpoint(
#             config_path,
#             model_path,
#             training=True,
#             trainable=True,
#             seq_len=MAX_SEQUENCE_LENGTH_D,
#         )
    
    layer_num = 12
    #with tf.get_default_graph().as_default():
#     model = load_trained_model_from_checkpoint(
#         config_path,
#         model_path,
#         training=False,
#         use_adapter=True,
#         seq_len=MAX_SEQUENCE_LENGTH_D,
#         trainable=['Encoder-{}-MultiHeadSelfAttention-Adapter'.format(i + 1) for i in range(layer_num)] +
#         ['Encoder-{}-FeedForward-Adapter'.format(i + 1) for i in range(layer_num)] +
#         ['Encoder-{}-MultiHeadSelfAttention-Norm'.format(i + 1) for i in range(layer_num)] +
#         ['Encoder-{}-FeedForward-Norm'.format(i + 1) for i in range(layer_num)],
#     )
    model = get_model(
        token_num=len(token_dict),
        head_num=5,
        transformer_num=layer_num,
        embed_dim=100,
        feed_forward_dim=100,
        seq_len=MAX_SEQUENCE_LENGTH_D,
        pos_num=MAX_SEQUENCE_LENGTH_D,
        dropout_rate=0.05,
    )
    compile_model(model)
    inputs = model.inputs[:2]
    outputs = model.get_layer('Encoder-{}-FeedForward-Norm'.format(layer_num)).output
    outputs = GlobalAveragePooling1D()(outputs)
    #outputs = Dense(300, activation='tanh')(outputs)
    
    model = Model(inputs, outputs, name='FeatureBERTGenerationModel')
    
    return model

### Bi-LSTM

In [38]:
from keras.layers import Dense, Input, LSTM, GRU, Dropout, Bidirectional, \
GlobalAveragePooling1D, Permute, Dot, TimeDistributed

def bilstm_model(embedding_layer, max_sequence_length):
    number_lstm_units = 50
    rate_drop_lstm = 0
    recurrent_dropout = 0

    sequence_input = Input(shape=(max_sequence_length, ), name='Feature_BugInput')
    #sequence_input = Input(shape=(None, ), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)

    left_layer = LSTM(number_lstm_units, return_sequences=True)(embedded_sequences)
    right_layer = LSTM(number_lstm_units, return_sequences=True, go_backwards=True)(left_layer)
    
    lstm_layer = Add()([left_layer, right_layer])
    
    layer = Flatten()(lstm_layer)
    #layer = GlobalAveragePooling1D()(layer)
    layer = Dense(300, activation='tanh')(layer)

    lstm_feature_model = Model(inputs=[sequence_input], outputs=[layer], name = 'FeatureLstmGenerationModel') # inputs=visible

    return lstm_feature_model

### MLP

In [39]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    #layer = GRU(100, activation='tanh')(layer)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [40]:
from keras import backend as K

'''
    Some loss ideas
    hinge loss Kullback-Leibler
    https://stackoverflow.com/questions/53581298/custom-combined-hinge-kb-divergence-loss-function-in-siamese-net-fails-to-genera
'''

def normalize(x, axis):
    norm = K.sqrt(K.sum(K.square(x), axis=axis, keepdims=False))
    return x, K.maximum(norm, K.epsilon())
    
# https://github.com/keras-team/keras/issues/3031
# https://github.com/keras-team/keras/issues/8335
def cosine_distance(inputs):
    x, y = inputs
    x, x_norm = normalize(x, axis=-1)
    y, y_norm = normalize(y, axis=-1)
    distance = K.sum( x * y, axis=-1) / (x_norm * y_norm)
    distance = (distance + K.constant(1)) / K.constant(2)
    # Distance goes from 0 to 2 in theory, but from 0 to 1 if x and y are both
    # positive (which is the case after ReLU activation).
    return K.mean(distance, axis=-1, keepdims=False)

def euclidean_distance(vects):
    x, y = vects
    distance = K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))
    # Normalize https://stats.stackexchange.com/questions/53068/euclidean-distance-score-and-similarity
    distance = K.constant(1) / (K.constant(1) + distance)
    return K.mean(distance, keepdims=False)
    #return K.mean(distance, axis=-1, keepdims=False)

# https://jdhao.github.io/2017/03/13/some_loss_and_explanations/
def triplet_loss(y_true, y_pred):
    margin = K.constant(1.0)
    pos = y_pred[0]
    neg = y_pred[1]
    return K.mean(K.maximum(0.0, pos - neg + margin))

def custom_margin_loss(y_true, y_pred):
    margin = K.constant(1.0)
    pos = y_pred[0]
    neg = y_pred[1]
    return K.mean(K.maximum(0.0, margin - pos + neg), keepdims=False)

# https://www.kaggle.com/c/quora-question-pairs/discussion/33631
# https://www.researchgate.net/figure/Illustration-of-triplet-loss-contrastive-loss-for-negative-samples-and-binomial_fig2_322060548
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    pos = y_pred[0]
    neg = y_pred[1]
    margin = 1
    return K.mean(pos * K.square(neg) +
                  (1 - pos) * K.square(K.maximum(margin - neg, 0)))

def pos_distance(y_true, y_pred):
    return y_pred[0]

def neg_distance(y_true, y_pred):
    return y_pred[1]

def stack_tensors(vects):
    return K.stack(vects, axis=-1)

### Propose

In [41]:
from keras.layers import concatenate, Add, Lambda, Average, Maximum, Subtract, Average, AveragePooling1D, GlobalAveragePooling1D
from keras.optimizers import Adam, Nadam

def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, name):
    
    bug_t = Input(shape = (sequence_length_t, ), name = 'title_{}'.format(name))
    bug_d_token = Input(shape = (sequence_length_d, ), name = 'desc_token_{}'.format(name))
    bug_d_segment = Input(shape = (sequence_length_d, ), name = 'desc_segment_{}'.format(name))
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    
    bug_t_feat = title_feature_model(bug_t)
    bug_d_feat = desc_feature_model([bug_d_token, bug_d_segment])
    bug_i_feat = categorical_feature_model(bug_i)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat], name = 'merge_features_{}'.format(name))
    
    bug_feature_model = Model(inputs=[bug_t, bug_d_token, bug_d_segment, bug_i], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [42]:
from keras.layers import Average
from keras_radam import RAdam
from keras_bert import AdamWarmup, calc_train_steps

def max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, 
                             master_anchor, master_negative, master_positive, 
                         NUMBER_OF_INSTANCES, BATCH_SIZE, EPOCHS, decay_lr=1):
    
    inputs = np.concatenate([encoded_anchor.input, encoded_positive.input, encoded_negative.input, 
                                 master_anchor.input, master_positive.input, master_negative.input], -1).tolist()
    
    encoded_anchor = encoded_anchor.output
    encoded_positive = encoded_positive.output
    encoded_negative = encoded_negative.output
    master_anchor = master_anchor.output
    master_negative = master_negative.output
    master_positive = master_positive.output
    
    # Distance bugs
    positive_d = Lambda(cosine_distance, name='pos_cosine_distance', output_shape=[1])([encoded_anchor, encoded_positive])
    negative_d = Lambda(cosine_distance, name='neg_cosine_distance', output_shape=[1])([encoded_anchor, encoded_negative])
    
    # Distance masters anchor
    master_anchor_positive_d = Lambda(cosine_distance, name='pos_master_cosine_distance', output_shape=[1])([encoded_anchor, master_positive])
    master_anchor_negative_d = Lambda(cosine_distance, name='neg_master_cosine_distance', output_shape=[1])([encoded_anchor, master_negative])
    
    # Distance master positive
    master_pos_positive_d = Lambda(cosine_distance, name='pos_master_pos_cosine_distance', output_shape=[1])([encoded_positive, master_positive])
    master_pos_negative_d = Lambda(cosine_distance, name='neg_master_pos_cosine_distance', output_shape=[1])([encoded_positive, master_negative])
    
    # Distance master negative
    master_neg_positive_d = Lambda(cosine_distance, name='pos_master_neg_cosine_distance', output_shape=[1])([encoded_negative, master_negative])
    master_neg_negative_d = Lambda(cosine_distance, name='neg_master_neg_cosine_distance', output_shape=[1])([encoded_negative, master_positive])
    

    # Loss function only works with a single output
    output_bug = Lambda(
        lambda vects: stack_tensors(vects),
        name='stack-distances-bug',
        output_shape=(2, 1)
    )([positive_d, negative_d])
    
    output_master = Lambda(
        lambda vects: stack_tensors(vects),
        name='stack-distances-master-anchor',
        output_shape=(2, 1)
    )([master_anchor_positive_d, master_anchor_negative_d])
    
    output_master_pos = Lambda(
        lambda vects: stack_tensors(vects),
        name='stack-distances-master-pos',
        output_shape=(2, 1)
    )([master_pos_positive_d, master_pos_negative_d])
    
    output_master_neg = Lambda(
        lambda vects: stack_tensors(vects),
        name='stack-distances-master-neg',
        output_shape=(2, 1)
    )([master_neg_positive_d, master_neg_negative_d])
    
    #output = Average()([output_bug, output_master, output_master_pos, output_master_neg])
    
    output_avg_master = Average()([output_master, output_master_pos, output_master_neg])
    output = Average()([output_bug, output_avg_master])
    #loss = MarginLoss()(output)

    similarity_model = Model(inputs = inputs, outputs = [output], name = 'Similarity_Model')

    #optimizer = Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.01)
    #optimizer = RAdam(lr=1e-10)
    LR = 1e-4
    decay_steps, warmup_steps = calc_train_steps(
        NUMBER_OF_INSTANCES,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
    )
    optimizer = AdamWarmup(decay_steps=decay_steps, warmup_steps=warmup_steps, lr=LR)

    # setup the optimization process 
    similarity_model.compile(optimizer=optimizer, loss=custom_margin_loss, 
                                 metrics=[pos_distance, neg_distance])

    return similarity_model

In [ ]:
#%%time
import os

'''
    Configuration
'''
epochs = 100
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

print("Batch size ", batch_size)

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# TF_KERAS must be added to environment variables in order to use TPU
#os.environ['TF_KERAS'] = '1'

# Embeddings
# desc_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
#                               num_words=len(baseline.embedding_matrix), 
#                               embedding_dim=EMBEDDING_DIM, 
#                               max_sequence_length=MAX_SEQUENCE_LENGTH_D, 
#                               trainable=False, name='desc')
title_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_T, 
                              trainable=False, name='title')

# Feature models
'''
    cnn_dilated_model
    arcii_model
    cnn_model
    lstm_model
    bilstm_model
'''
title_feature_model = bilstm_model(title_embedding_layer, MAX_SEQUENCE_LENGTH_T)
desc_feature_model = bert_model(MAX_SEQUENCE_LENGTH_D)
#desc_feature_model = cnn_model(desc_embedding_layer, MAX_SEQUENCE_LENGTH_D)
categorical_feature_model = mlp_model(number_of_columns_info)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'in')
encoded_positive = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'pos')
encoded_negative = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'neg')
# Master model
master_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'master_in')
master_positive = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'master_pos')
master_negative = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'master_neg')

NUMBER_OF_INSTANCES = len(baseline.dup_sets_train)
BATCH_SIZE = batch_size
EPOCHS = epochs

similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, 
                                            master_anchor, master_negative, master_positive,
                                            NUMBER_OF_INSTANCES, BATCH_SIZE, EPOCHS, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()

'''
    Experiment
'''
for epoch in range(epochs):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, train_master_input, train_master_neg, \
            train_sim = batch_iterator(baseline, baseline.train_data, baseline.dup_sets_train, bug_train_ids, 
                                       batch_size, 1, issues_by_buckets)
    
    train_batch = [train_input_sample['title'], train_input_sample['description']['token'], train_input_sample['description']['segment'], train_input_sample['info'],
                   train_input_pos['title'], train_input_pos['description']['token'], train_input_pos['description']['segment'], train_input_pos['info'], 
                   train_input_neg['title'], train_input_neg['description']['token'], train_input_neg['description']['segment'], train_input_neg['info'],
                  train_master_input['title'], train_master_input['description']['token'], train_master_input['description']['segment'], train_master_input['info'],
                  train_master_input['title'], train_master_input['description']['token'], train_master_input['description']['segment'], train_master_input['info'],
                   train_master_neg['title'], train_master_neg['description']['token'], train_master_neg['description']['segment'], train_master_neg['info']]
    
#     if epoch == 10:
#         similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=0.1)
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, _, debug = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, 
                                                               bug_train_ids, method='bert')
        print("Epoch: {} Loss: {:.2f}, pos_cosine: {:.2f}, neg_cosine: {:.2f}, recall@25: {:.2f}".format(epoch+1,
                                                                                                         h[0],
                                                                                                         h[1], h[2], recall))
    else:
        print("Epoch: {} Loss: {:.2f}, pos_cosine: {:.2f}, neg_cosine: {:.2f}".format(epoch+1,
                                                                                                         h[0],
                                                                                                         h[1],
                                                                                                         h[2]))
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}s, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 1682)         0                                            
__________________________________________________________________________________________________
title_in (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 100)          0                                            
__________________________________________________________________________________________________
desc_segment_in (InputLayer)    (None, 100)          0                                            
__________________________________________________________________________________________________
info_pos (

Epoch: 1 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 2 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 3 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 4 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 5 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 6 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 7 Loss: 1.00, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 8 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 9 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 10 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 11 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 12 Loss: 1.00, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 13 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 14 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 15 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 16 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 17 Loss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 18 Loss: 0.99, pos_cosine: 0.98, 

In [ ]:
recall

In [ ]:
_[:20]

In [ ]:
# recall, exported_rank, debug = experiment.evaluate_validation_test(experiment, retrieval, verbose, 
#                                                         encoded_anchor, issues_by_buckets, evaluate_validation_test)
# test_vectorized, queries_test_vectorized, annoy, X_test, distance_test, indices_test = debug
# "recall@25 last epoch:", recall

### Retrieval evaluation

In [ ]:
print("Total of queries:", len(retrieval.test))

#### Getting the model trained

In [ ]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

In [ ]:
model = encoded_anchor
# model = experiment.get_model_vectorizer(path=SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)))

In [ ]:
model.summary()

In [ ]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, 0, model, issues_by_buckets, 
                                                                   bug_train_ids, method='bert')

In [ ]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

In [ ]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [ ]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report